Automatic trash emptying: https://www.amixa.com/blog/2019/04/05/automatically-empty-google-drive-trash/

In [1]:
!pip install OpenNMT-py

     |████████████████████████████████| 184kB 6.8MB/s 
     |████████████████████████████████| 2.2MB 37.5MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-0.15.1-cp36-none-any.whl size=17764 sha256=de5baa9f2baeb585ebfdb3d09472f795c8e46b0ef4ae1b215feccbc1da3cc0b3
  Stored in directory: /root/.cache/pip/wheels/fc/01/53/c6db85bbdf686b64c0f81bf78d16465455e8cda115e4c5264b
Successfully built configargparse
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [1]:
!pip install torch==1.2.0 torchvision==0.4.0
import torch
print(torch.__version__)

     |████████████████████████████████| 748.9MB 21kB/s 
     |████████████████████████████████| 8.8MB 22.1MB/s 
  Found existing installation: torch 1.3.0+cu100
    Uninstalling torch-1.3.0+cu100:
      Successfully uninstalled torch-1.3.0+cu100
  Found existing installation: torchvision 0.4.1+cu100
    Uninstalling torchvision-0.4.1+cu100:
      Successfully uninstalled torchvision-0.4.1+cu100
1.2.0


In [0]:
# preprocessing
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/My Drive/University/MAIS202/Final Project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/University/MAIS202/Final Project


In [0]:
import os
import re
import pandas as pd
import numpy as np
import itertools as it

In [0]:
def compute_accuracy(filename_pred, filename_tgt):
    
    print('Computing accuracy of predictions in {}...'.format(filename_pred))

    with open(filename_pred) as file_pred, open(filename_tgt) as file_tgt:

        count = 0
        correct = 0

        for pred, tgt in zip(file_pred, file_tgt):

            count += 1

            if pred == tgt:
                correct += 1

    return correct/count

def compute_bleu():

    !perl OpenNMT-py/tools/multi-bleu.perl data/tgt-test.txt < ${PATH_PREDS}/${PRED_FILE} > bleu.txt

    with open('bleu.txt') as file_bleu:
        for line in file_bleu:
            bleu = re.search('BLEU = (\d+.\d+)', line).group(1)
            return bleu

In [0]:
def keep_best_checkpoints(path_model, logfile, metric='acc', n_best=5):
    
    _, checkpoints_to_keep = get_best_models(path_model, logfile, n=n_best)
    print(checkpoints_to_keep)

    checkpoints_regex = re.compile("|".join(checkpoints_to_keep))

    for f in os.listdir(path_model):
        if f.endswith('.pt') and not re.search(checkpoints_regex, f):
            os.remove(os.path.join(path_model, f))
            # print('removing {}'.format(os.path.join(path_model, f)))
            # os.rename(os.path.join(path_model, f), os.path.join('to_delete', f))
    
    # file_list = my_drive.ListFile({'q': "'1YP97oObrJjb6QLOT4roVCQLsdW3DNpfv' in parents and trashed=false"}).GetList()
    # print([file_list[i]['title'] for i in range(np.size(file_list))])

    # for i in range(np.size(file_list)):
    #     file_name = file_list[i]['title']
    #     my_drive.CreateFile({'id': file_list[i]['id']}).Delete()

def get_best_models(path_model, logfile, metric='acc', n=5):

    with open(os.path.join(path_model, logfile)) as file_log:
        acc_list = []
        ppl_list = []
        checkpoint_steps_list = []

        for line in file_log:

            acc = re.search('Validation accuracy: (\d+.\d+)', line)
            ppl = re.search('Validation perplexity: (\d+.\d+)', line)
            checkpoint_steps = re.search('Saving checkpoint .+_step_(\d+).pt', line)

            if acc:
                acc_list.append(acc.group(1))

            if ppl:
                ppl_list.append(ppl.group(1))

            if checkpoint_steps:
                checkpoint_steps_list.append(checkpoint_steps.group(1))

        acc_list = np.array(acc_list)
        ppl_list = np.array(ppl_list)
        checkpoint_steps_list = np.array(checkpoint_steps_list)

        if metric == 'acc':
            if not len(acc_list) == len(checkpoint_steps_list):
                print('Error: acc_list and checkpoint_steps_list do not have the same number of elements')
                print(len(acc_list))
                print(len(checkpoint_steps_list))
                return

            id_sort = np.argsort(acc_list)

        elif metric == 'ppl':
            if not len(ppl_list) == len(checkpoints_steps_list):
                print('Error: ppl_list and checkpoint_steps_list do not have the same number of elements')
                print(ppl_list)
                print(checkpoint_steps_list)
                return

            id_sort = np.argsort(ppl_list)

        # print(acc_list[id_sort])
        # print(ppl_list[id_sort])
        # print(checkpoint_steps_list[id_sort])

        best_checkpoints = np.flip(checkpoint_steps_list[id_sort[-n:]])

        # print('best models at these steps: {}'.format(best_checkpoints))

        best_models = [os.path.join(path_model, 'model{}_step_{}.pt'.format(model_id, s)) for s in best_checkpoints]

        # print(best_models)

    print(best_models, best_checkpoints)
    return best_models, best_checkpoints

# model_id = 0
# keep_best_checkpoints('models/model0', 'model0-training-log.txt')

In [0]:
# !onmt_preprocess -train_src data/src-train.txt -train_tgt data/tgt-train.txt -valid_src data/src-val.txt -valid_tgt data/tgt-val.txt -save_data data/processed_data

In [0]:
# hp_filename = 'hyperparameters.csv'
# df = pd.read_csv(hp_filename, index_col=0)
# model_id = len(df.index)
# # df.loc[model_id] = ['model0-training-log.txt','rnn',2,500,0.3,64,10,'sgd','model0-training-log.txt','model0_steps_96900.pt', 0.6573108450281104, 80.41]
# df.head()

,encoder,layers,rnn_size,dropout,batch_size,patience,optimizer,logfile,best_model,accuracy,bleu
0,rnn,2,500,0.3,64,10,sgd,model0-training-log.txt,model0_steps_96900.pt,0.657311,80.41
1,rnn,2,500,0.2,32,10,adam,model1-training-log.txt,model1_steps_101960.pt,0.646625,79.79


In [0]:
learning_rates = {'adam':0.001, 'sgd':1, 'adagrad':0.1, 'adadelta':1} # recommended learning rates in OpenNMT-py documentation
n_samples = 81551

encoder_all = ['brnn']
layers_all = [2,3,4]
rnn_size_all = [500,600] # if 1000, checkpoint files huge (402 Mb)... Maybe try with sgd instead of adam?
dropout_all = [0.1,0.2,0.3]
batch_size_all = [32]
patience_all = [10]
optimizer_all = ['adam']
hyperparameters_all = it.product(optimizer_all, patience_all, batch_size_all, dropout_all, rnn_size_all, layers_all, encoder_all)

for hyperparameters in hyperparameters_all:

    # load df
    hp_filename = 'hyperparameters.csv'
    df = pd.read_csv(hp_filename, index_col=0)

    # get id for next model
    model_id = len(df.index)

    optimizer, patience, batch_size, dropout, rnn_size, layers, encoder = hyperparameters

    # skip to next combination if model already exists
    df_temp = df.loc[(df['encoder'] == encoder) & (df['layers'] == layers) & (df['rnn_size'] == rnn_size) &
                     (df['dropout'] == dropout) & (df['batch_size'] == batch_size) & (df['optimizer'] == optimizer)]
    if not len(df_temp) == 0:
        print('Skipping this set of hyperparameters: {}'.format(hyperparameters))
        continue

    print('Hyperparameters: {}, {}, {}, {}, {}, {}, {}'.format(optimizer, patience, batch_size, dropout, rnn_size, layers, encoder))

    # set up path and filename variables
    path_model = 'models/model{}'.format(model_id)
    path_data = 'data/processed_data'
    path_preds = 'predictions'

    log_file = 'model{}-training-log.txt'.format(model_id)
    pred_file = 'model{}-preds.txt'.format(model_id)

    # set up shell environmental variables
    os.environ['MODEL_ID'] = str(model_id)

    os.environ['PATH_MODEL'] = path_model
    os.environ['PATH_DATA'] = path_data
    os.environ['PATH_PREDS'] = path_preds
    os.environ['LOGFILE'] = log_file
    os.environ['PREDFILE'] = pred_file

    os.environ['ENCODER'] = encoder
    os.environ['LAYERS'] = str(layers)
    os.environ['RNN_SIZE'] = str(rnn_size)
    os.environ['DROPOUT'] = str(dropout)
    os.environ['BATCH_SIZE'] = str(batch_size)
    os.environ['STEPS_PER_EPOCH'] = str(n_samples//batch_size + 1)
    os.environ['TRAIN_STEPS'] = str(75 * (n_samples//batch_size + 1))
    os.environ['PATIENCE'] = str(patience)
    os.environ['OPTIM'] = optimizer
    os.environ['LEARNING_RATE'] = str(learning_rates[optimizer])

    # make directory for new model
    !mkdir $PATH_MODEL

    # train
    !onmt_train -gpu_ranks 0 \
        -data $PATH_DATA \
        -train_steps $TRAIN_STEPS \
        -report_every $STEPS_PER_EPOCH \
        -log_file ${PATH_MODEL}/$LOGFILE -save_model ${PATH_MODEL}/model${MODEL_ID} \
        -encoder_type $ENCODER \
        --layers $LAYERS \
        -rnn_size $RNN_SIZE \
        -dropout $DROPOUT \
        --batch_size $BATCH_SIZE \
        -valid_steps $STEPS_PER_EPOCH -save_checkpoint_steps $STEPS_PER_EPOCH \
        --early_stopping $PATIENCE \
        -early_stopping_criteria accuracy \
        -optim $OPTIM -learning_rate $LEARNING_RATE \
        -keep_checkpoint 20

    best_models, best_model_steps = get_best_models(path_model, log_file, n=1)

    os.environ['BEST_MODEL'] = best_models[0]
    os.environ['PRED_FILE'] = pred_file
    print(best_models[0], pred_file)

    # translate
    ! onmt_translate \
        -model ${BEST_MODEL} \
        -src data/src-test.txt \
        -output ${PATH_PREDS}/${PRED_FILE} \
        --beam_size 5

    # evaluate
    bleu = compute_bleu()
    accuracy = compute_accuracy(os.path.join(path_preds, pred_file), 'data/tgt-test.txt')

    print('BLEU:', bleu)
    print('Accuracy:', accuracy)

    # update and save hyperparameters file
    df.loc[model_id] = [encoder, layers, rnn_size, dropout, batch_size, patience, optimizer, log_file,
                        'model{}_steps_{}.pt'.format(model_id, best_model_steps[0]), accuracy, bleu]
    df.to_csv(hp_filename)

    # delete all but 5 best models
    keep_best_checkpoints(path_model, log_file)
    

Hyperparameters: adam, 10, 32, 0.1, 500, 2, brnn
[2019-11-09 15:26:43,408 INFO]  * src vocab size = 28
[2019-11-09 15:26:43,409 INFO]  * tgt vocab size = 73
[2019-11-09 15:26:43,409 INFO] Building model...
[2019-11-09 15:26:48,467 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(28, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 250, num_layers=2, dropout=0.1, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(73, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.1, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Li

In [10]:
# # model4 retraining

# n_samples = 81551

# model_id = 4

# encoder = 'brnn'
# layers = 3
# rnn_size = 500
# dropout = 0.2
# batch_size = 32
# patience = 10
# optimizer = 'adam'

# print('Hyperparameters: {}, {}, {}, {}, {}, {}, {}'.format(optimizer, patience, batch_size, dropout, rnn_size, layers, encoder))

# # set up path and filename variables
# path_model = 'models/model{}'.format(model_id)
# path_data = 'data/processed_data'
# path_preds = 'predictions'

# log_file = 'model{}-training-log.txt'.format(model_id)
# pred_file = 'model{}-preds.txt'.format(model_id)

# # set up shell environmental variables
# os.environ['MODEL_ID'] = str(model_id)

# os.environ['PATH_MODEL'] = path_model
# os.environ['PATH_DATA'] = path_data
# os.environ['PATH_PREDS'] = path_preds
# os.environ['LOGFILE'] = log_file
# os.environ['PREDFILE'] = pred_file

# os.environ['ENCODER'] = encoder
# os.environ['LAYERS'] = str(layers)
# os.environ['RNN_SIZE'] = str(rnn_size)
# os.environ['DROPOUT'] = str(dropout)
# os.environ['BATCH_SIZE'] = str(batch_size)
# os.environ['STEPS_PER_EPOCH'] = str(n_samples//batch_size + 1)
# os.environ['TRAIN_STEPS'] = str(75 * (n_samples//batch_size + 1))
# os.environ['PATIENCE'] = str(patience)
# os.environ['OPTIM'] = optimizer
# os.environ['LEARNING_RATE'] = str(learning_rates[optimizer])

# # train
# !onmt_train -gpu_ranks 0 \
#     -data $PATH_DATA \
#     -train_steps $TRAIN_STEPS \
#     -report_every $STEPS_PER_EPOCH \
#     -log_file ${PATH_MODEL}/$LOGFILE -save_model ${PATH_MODEL}/model${MODEL_ID} \
#     --batch_size $BATCH_SIZE \
#     -valid_steps $STEPS_PER_EPOCH -save_checkpoint_steps $STEPS_PER_EPOCH \
#     --early_stopping $PATIENCE \
#     -early_stopping_criteria accuracy \
#     -learning_rate $LEARNING_RATE \
#     -keep_checkpoint 20 \
#     -train_from models/model4/model4_step_122352.pt

Hyperparameters: adam, 10, 32, 0.2, 500, 3, brnn
[2019-11-10 03:56:06,550 INFO] Loading checkpoint from models/model4/model4_step_122352.pt
[2019-11-10 03:56:07,102 INFO] Loading vocab from checkpoint at models/model4/model4_step_122352.pt.
[2019-11-10 03:56:07,103 INFO]  * src vocab size = 28
[2019-11-10 03:56:07,103 INFO]  * tgt vocab size = 73
[2019-11-10 03:56:07,104 INFO] Building model...
[2019-11-10 03:56:09,698 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(28, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 250, num_layers=3, dropout=0.2, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(73, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): StackedLSTM(
      (dropo

In [11]:
# translate and evaluate

# model_id = 
# path_model = 'models/model{}'.format(model_id)
# path_preds = 'predictions'
# log_file = 'model{}-training-log.txt'.format(model_id)
# pred_file = 'model{}-preds.txt'.format(model_id)

# # MAKE SURE THESE ARE CORRECT
# # Hyperparameters: adam, 10, 32, 0.2, 600, 3, brnn
# # encoder = 'brnn'
# # layers = 3
# # rnn_size = 500
# # dropout = 0.2
# # batch_size = 32
# # patience = 10
# # optimizer = 'adam'

# hp_filename = 'hyperparameters.csv'
# df = pd.read_csv(hp_filename, index_col=0)

# best_models, best_model_steps = get_best_models(path_model, log_file, n=1)

# os.environ['BEST_MODEL'] = best_models[0]
# os.environ['PATH_PREDS'] = path_preds
# os.environ['PRED_FILE'] = pred_file
# print(best_models[0], pred_file)

# # translate
# ! onmt_translate \
#     -model ${BEST_MODEL} \
#     -src data/src-test.txt \
#     -output ${PATH_PREDS}/${PRED_FILE} \
#     --beam_size 5

# # evaluate
# bleu = compute_bleu()
# accuracy = compute_accuracy(os.path.join(path_preds, pred_file), 'data/tgt-test.txt')

# print('BLEU:', bleu)
# print('Accuracy:', accuracy)

# # update and save hyperparameters file
# df.loc[model_id] = [encoder, layers, rnn_size, dropout, batch_size, patience, optimizer, log_file,
#                     'model{}_steps_{}.pt'.format(model_id, best_model_steps[0]), accuracy, bleu]
# df.to_csv(hp_filename)

# # delete all but 5 best models
# keep_best_checkpoints(path_model, log_file)

['models/model4/model4_step_124901.pt'] ['124901']
models/model4/model4_step_124901.pt model4-preds.txt
[2019-11-10 04:30:43,964 INFO] Translating shard 0.
PRED AVG SCORE: -0.0165, PRED PPL: 1.0166
[2019-11-10 04:33:32,342 INFO] Translating shard 1.
PRED AVG SCORE: -0.0169, PRED PPL: 1.0170
[2019-11-10 04:36:21,168 INFO] Translating shard 2.
PRED AVG SCORE: -0.0167, PRED PPL: 1.0169
Computing accuracy of predictions in predictions/model4-preds.txt...
BLEU: 81.35
Accuracy: 0.6679970816703146
['models/model4/model4_step_124901.pt', 'models/model4/model4_step_140195.pt', 'models/model4/model4_step_129999.pt', 'models/model4/model4_step_127450.pt', 'models/model4/model4_step_122352.pt'] ['124901' '140195' '129999' '127450' '122352']
['124901' '140195' '129999' '127450' '122352']
